# Load libraries and URMs

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
import scipy.sparse as sps
from tqdm import tqdm
import scipy.sparse as sps
import numpy as np
from xgboost import XGBRanker
import os

import Utils.DataReader as dr
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

URM_all = dr.load_urm("../data/URMtries/RatingsBinary.csv")
ICM_tr = dr.load_icm("../data/data_ICM_type_truncated.csv")

URM_train = sps.load_npz("trainDefault.npz")
URM_valid = sps.load_npz("validDefault.npz")
stacked_URM = sps.vstack([URM_all, ICM_tr.T])
stacked_URM_train = sps.vstack([URM_train, ICM_tr.T])
evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])

C:\Users\Andrea\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


EvaluatorHoldout: Ignoring 963 ( 2.3%) Users that have less than 1 test interactions


# Create the best hybrid

In [200]:
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Recommenders.Recommender_utils import check_matrix, similarityMatrixTopK

class MergeSimilarity4(BaseItemSimilarityMatrixRecommender):
    """ MergeSimilarity3
    Hybrid of two similarities S = S1*alpha + S2*beta + S3*gamma

    """

    RECOMMENDER_NAME = "ItemKNNSimilarityHybridRecommender"

    def __init__(self, URM_train, Similarity_1, Similarity_2, Similarity_3, Similarity_4, verbose=True):
        super(MergeSimilarity4, self).__init__(URM_train, verbose=verbose)

        self.W_sparse = None

        if Similarity_1.shape != Similarity_2.shape:
            raise ValueError(
                "ItemKNNSimilarityHybridRecommender: similarities have different size, S1 is {}, S2 is {}".format(
                    Similarity_1.shape, Similarity_2.shape
                ))

        if Similarity_1.shape != Similarity_3.shape:
            raise ValueError(
                "ItemKNNSimilarityHybridRecommender: similarities have different size, S1 is {}, S3 is {}".format(
                    Similarity_1.shape, Similarity_3.shape
                ))

        if Similarity_2.shape != Similarity_3.shape:
            raise ValueError(
                "ItemKNNSimilarityHybridRecommender: similarities have different size, S2 is {}, S3 is {}".format(
                    Similarity_2.shape, Similarity_3.shape
                ))

        # CSR is faster during evaluation
        self.Similarity_1 = check_matrix(Similarity_1.copy(), 'csr')
        self.Similarity_2 = check_matrix(Similarity_2.copy(), 'csr')
        self.Similarity_3 = check_matrix(Similarity_3.copy(), 'csr')
        self.Similarity_4 = check_matrix(Similarity_4.copy(), 'csr')


    def fit(self, topK=100, alpha=0.5, beta=0.5, gamma=0.5, delta=0.5):

        self.topK = topK
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta

        W_sparse = self.Similarity_1 * self.alpha + self.Similarity_2 * self.beta + self.Similarity_3 * self.gamma + self.Similarity_4 * self.delta

        self.W_sparse = similarityMatrixTopK(W_sparse, k=self.topK)
        self.W_sparse = check_matrix(self.W_sparse, format='csr')


# %%
from Recommenders.BaseRecommender import BaseRecommender


class DifferentLossScoresHybridRecommender(BaseRecommender):

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        self.preferences = pd.read_csv("./data/user_desc.csv")
        self.preferences = self.preferences.drop(columns=["interactions", "avg_len"])
        self.ICM_tr = dr.load_icm("../data/data_ICM_type_truncated.csv").todense()

    def fit(self, alpha=0.5, beta=0.5, gamma=0.5):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma

    def _compute_item_score(self, user_id_array, items_to_compute):
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        item_weights = item_weights_1 * self.alpha + item_weights_2 * self.beta + item_weights_3 * self.gamma
        #print(item_weights.shape) #len(user_id_array) x len(items)
        #print("user_id_array: ", user_id_array) #user id

        for user_index in range(len(user_id_array)):
            user_id = user_id_array[user_index]
            #ICM_tr.todense()[2].argmax() #type of item 2 (type is 4)
            #preferences[preferences["iduser"] == 0].values.flatten() #preferences of user 0
            #considering only the 20 best item scores
            #print("user: ", user_id)
            #best_scores_indices_unsorted = np.argpartition(item_weights[user_index], -20)[-20:]
            #best_scores_values_unsorted = item_weights[user_index][best_scores_indices_unsorted]
            #print("best_scores indices unsorted: ")
            #print(best_scores_indices_unsorted)
            best_scores_indices = np.argsort(item_weights[user_index])[0:200]
            #print("indices: ")
            #print(best_scores_indices)
            #print("values")
            #print(item_weights[user_index][best_scores_indices])
            for item_index in best_scores_indices:
                item_type = self.ICM_tr[item_index].argmax()
                #print("item type is ", item_type)
                #print("item weight [i] is ", item_weights[user_index][item_index])
                #print("this user's preferences:")
                #print(self.preferences[preferences["iduser"] == user_id])
                #print("user preference for this type is ", self.preferences[preferences["iduser"] == user_id].values.flatten()[item_type])
                item_weights[user_index][item_index] += item_weights[user_index][item_index] * self.preferences[preferences["iduser"] == user_id].values.flatten()[item_type]


        return item_weights

In [3]:
recommender_EASE_all = EASE_R_Recommender(stacked_URM)
recommender_EASE_all.fit(topK=385, l2_norm=138.06756005486739, normalize_matrix=False)

EASE_R_Recommender: URM Detected 3 ( 0.0%) users with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 4.93 min


In [4]:
recommender_SLIMElasticNet_all = SLIMElasticNetRecommender(stacked_URM)
recommender_SLIMElasticNet_all.load_model(
    folder_path= "C:\\Users\\Andrea\\AppData\\Roaming\\JetBrains\\DataSpell2022.2\\projects\\RecSys2022\\data\\saved_models",
    file_name= recommender_SLIMElasticNet_all.RECOMMENDER_NAME + "-allStack"
)
recommender_SLIMElasticNet_all.W_sparse = recommender_SLIMElasticNet_all.W_sparse.get('W_sparse')

SLIMElasticNetRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SLIMElasticNetRecommender: Loading model from file 'C:\Users\Andrea\AppData\Roaming\JetBrains\DataSpell2022.2\projects\RecSys2022\data\saved_modelsSLIMElasticNetRecommender-allStack'
  (0, 6)	0.002692566
  (0, 18)	0.0055840663
  (0, 90)	0.0024054425
  (0, 135)	0.0037545408
  (0, 149)	0.0076460163
  (0, 171)	0.004285143
  (0, 212)	0.008502655
  (0, 213)	0.005583007
  (0, 244)	0.007311898
  (0, 305)	0.0024187043
  (0, 306)	0.019745024
  (0, 315)	0.0012861125
  (0, 320)	0.001745392
  (0, 328)	0.009700223
  (0, 329)	0.0019916082
  (0, 350)	0.0033767025
  (0, 380)	0.0011122178
  (0, 391)	0.003471503
  (0, 441)	0.00081586116
  (0, 472)	0.0011776311
  (0, 488)	0.006773121
  (0, 491)	0.0008619133
  (0, 533)	0.004399461
  (0, 560)	0.007614849
  (0, 623)	0.0007331659
  :	:
  (24506, 23540)	0.004995775
  (24506, 23562)	0.0020068493
  (24506, 23594)	0.0066600703
  (24506, 23823)	0.0031086593
  (24506, 23863)	0.004972873


In [5]:
recommender_RP3_all = RP3betaRecommender(URM_all)
recommender_RP3_all.fit(alpha=1.0, beta=0.3168683362512345, topK=50, implicit=False)

RP3betaRecommender: Similarity column 24507 (100.0%), 3334.66 column/sec. Elapsed time 7.35 sec


In [6]:
recommenderSlimBpr_all = SLIM_BPR_Cython(URM_all)
recommenderSlimBpr_all.fit(epochs=1230, sgd_mode = "adagrad", symmetric= True, topK = 100,
                           lambda_i = 5e-05, lambda_j = 0.00012058123924534603, learning_rate = 0.1)

Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 41629 (100.0%) in 0.81 sec. BPR loss is 9.04E-01. Sample per second: 51333
SLIM_BPR_Recommender: Epoch 1 of 1230. Elapsed time 0.24 sec
Processed 41629 (100.0%) in 1.11 sec. BPR loss is 1.96E+00. Sample per second: 37574
SLIM_BPR_Recommender: Epoch 2 of 1230. Elapsed time 0.53 sec
Processed 41629 (100.0%) in 0.38 sec. BPR loss is 2.62E+00. Sample per second: 109686
SLIM_BPR_Recommender: Epoch 3 of 1230. Elapsed time 0.80 sec
Processed 41629 (100.0%) in 0.64 sec. BPR loss is 3.02E+00. Sample per second: 65126
SLIM_BPR_Recommender: Epoch 4 of 1230. Elapsed time 1.06 sec
Processed 41629 (100.0%) in 0.91 sec. BPR loss is 3.50E+00. Sample per second: 45723
SLIM_BPR_Recommender: Epoch 5 of 1230. Elapsed time 1.33 sec
Processed 41629 (100.0%) in 1.16 sec. BPR loss is

In [7]:
recommender_h9_all = MergeSimilarity4(URM_train= stacked_URM, Similarity_1= recommender_SLIMElasticNet_all.W_sparse,Similarity_2= sps.csr_matrix(recommenderSlimBpr_all.W_sparse), Similarity_3= recommender_RP3_all.W_sparse,Similarity_4= sps.csr_matrix(recommender_EASE_all.W_sparse))
recommender_h9_all.fit(alpha= 100, beta= 0.8045044736114458, gamma= 44.580548079262385, delta= 7.692770938311851, topK= 1500)

ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.


In [8]:
recommender_IALS_all = IALSRecommender(URM_train)
recommender_IALS_all.load_model(
    folder_path= "C:\\Users\\Andrea\\AppData\\Roaming\\JetBrains\\DataSpell2022.2\\projects\\RecSys2022\\data\\saved_models",
    file_name= recommender_IALS_all.RECOMMENDER_NAME + "-all"
)

IALSRecommender: Loading model from file 'C:\Users\Andrea\AppData\Roaming\JetBrains\DataSpell2022.2\projects\RecSys2022\data\saved_modelsIALSRecommender-all'
{'use_bias': False}
{'use_bias': False, 'ITEM_factors': array([[ 0.0338902 ,  0.04183504,  0.03235699, ...,  0.04818966,
         0.05181339,  0.0394293 ],
       [ 0.04026608, -0.05633232, -0.01656853, ...,  0.10920366,
         0.0805305 ,  0.08195469],
       [ 0.04639078,  0.0505322 ,  0.03300728, ...,  0.00436769,
         0.0800997 ,  0.03589458],
       ...,
       [ 0.0448172 ,  0.04220368, -0.00141579, ..., -0.01604275,
         0.03158257,  0.01223   ],
       [-0.02040121,  0.02083441,  0.02736818, ...,  0.01201205,
         0.02841607, -0.01189747],
       [ 0.02354587,  0.01858865,  0.01174355, ...,  0.0293125 ,
         0.03748057,  0.00074187]])}
{'use_bias': False, 'ITEM_factors': array([[ 0.0338902 ,  0.04183504,  0.03235699, ...,  0.04818966,
         0.05181339,  0.0394293 ],
       [ 0.04026608, -0.05633232, -0

In [9]:
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_FunkSVD_Cython
recommender_FunkSVD_all = MatrixFactorization_FunkSVD_Cython(URM_train)
recommender_FunkSVD_all.load_model(
    folder_path= "C:\\Users\\Andrea\\AppData\\Roaming\\JetBrains\\DataSpell2022.2\\projects\\RecSys2022\\data\\saved_models",
    file_name= recommender_FunkSVD_all.RECOMMENDER_NAME + "-all"
)

MatrixFactorization_FunkSVD_Cython_Recommender: Loading model from file 'C:\Users\Andrea\AppData\Roaming\JetBrains\DataSpell2022.2\projects\RecSys2022\data\saved_modelsMatrixFactorization_FunkSVD_Cython_Recommender-all'
{'ITEM_bias': array([-0.05554075, -0.03800917, -0.0516378 , ..., -0.05234976,
       -0.05898234, -0.05799737])}
{'ITEM_bias': array([-0.05554075, -0.03800917, -0.0516378 , ..., -0.05234976,
       -0.05898234, -0.05799737]), 'use_bias': True}
{'ITEM_bias': array([-0.05554075, -0.03800917, -0.0516378 , ..., -0.05234976,
       -0.05898234, -0.05799737]), 'use_bias': True, 'ITEM_factors': array([[-0.05031871, -0.06147257,  0.05322883, ..., -0.05059848,
         0.11844476, -0.14673261],
       [-0.17657504, -0.02729923, -0.11592639, ..., -0.22684507,
        -0.16970729, -0.05463818],
       [ 0.12432   , -0.19350822, -0.12574418, ...,  0.00743124,
        -0.03476391, -0.01874313],
       ...,
       [ 0.05202964, -0.1577066 , -0.00925799, ..., -0.1567929 ,
        -0.1

In [201]:
recommender_h11 = DifferentLossScoresHybridRecommender(URM_train= stacked_URM_train, recommender_1= recommender_h9_all, recommender_2= recommender_FunkSVD_all, recommender_3= recommender_IALS_all)
recommender_h11.fit(alpha=  16.78817603196424, beta= 67.64890146022836, gamma= 125.2779343914485)

DifferentLossScoresHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.


In [150]:
#recommender_h11.recommend(user_id_array=[0,1,2])
ICM_tr.todense()[2].argmax() #type of item 2 (type is 4)

4

# Load user preferences

In [153]:
preferences = pd.read_csv("./data/user_desc.csv")
preferences = preferences.drop(columns=["interactions", "avg_len"])

In [157]:
type(preferences[preferences["iduser"] == 0].values.flatten()[4]) #array of user0's preferences

numpy.float64

# Rerank

In [91]:
test_users = pd.read_csv('./data/data_target_users_test.csv')
user_id = test_users['user_id']
recommendations = []
item_scores = []
for user in user_id:
    rec, score = recommender_h11.recommend(user, cutoff=15, return_scores=True)
    recommendations.append(rec)
    item_scores.append(score)

In [95]:
np_scores = np.array(item_scores)
np_scores.shape

(41116, 1, 24507)

In [96]:
np_recs = np.array(recommendations)
np_recs.shape

(41116, 15)

In [104]:
np_scores

array([[[-2.83313827e+00,  1.19741751e+02,  3.73435981e+01, ...,
          1.34968914e+01,  1.19700375e+01,  2.57516887e+01]],

       [[ 4.08416781e+00, -8.38891299e+00,  9.71844093e+00, ...,
         -3.74169495e+00, -9.21797435e+00,  1.84210805e+00]],

       [[ 2.01539121e+01,  1.00362434e+02,  5.85825061e+01, ...,
         -2.52744541e+00,  1.28447415e+01,  6.38857755e-03]],

       ...,

       [[ 3.95200344e-01,  2.07451536e+01,  7.36102701e+00, ...,
          3.35271042e+00,  3.92354873e-01,  2.36591661e+01]],

       [[-1.18201214e+00,  2.14270904e+01,  1.77436388e+01, ...,
          4.62177325e+00,  3.25702774e+01,  1.26910342e+01]],

       [[ 1.11094196e+01,  5.42387659e+01,  1.08160977e+01, ...,
          7.57029544e+00,  4.21289296e+00,  5.06826082e+00]]])

In [97]:
np_recs

array([[  617,   752,    20, ...,  2494, 14710,   519],
       [ 6874,   665,    62, ..., 14687,    60, 20203],
       [  121,  7864,   752, ...,   925, 22631,   750],
       ...,
       [   56,    21,    53, ...,  1691,    23,   445],
       [ 1214,   126,  1859, ...,    71,  5136,    95],
       [  769,   771,   767, ...,  2512,  6967,   207]])

In [89]:
#recommender_h11.recommend(0, cutoff=15, return_scores=False)

[617, 752, 20, 391, 18484, 124, 2488, 107, 29, 23, 673, 2495, 2494, 14710, 519]

In [59]:
reshaped_scores = np_scores.reshape(41116, 24507)
reshaped_scores

array([[-2.83313827e+00,  1.19741751e+02,  3.73435981e+01, ...,
         1.34968914e+01,  1.19700375e+01,  2.57516887e+01],
       [ 4.08416781e+00, -8.38891299e+00,  9.71844093e+00, ...,
        -3.74169495e+00, -9.21797435e+00,  1.84210805e+00],
       [ 2.01539121e+01,  1.00362434e+02,  5.85825061e+01, ...,
        -2.52744541e+00,  1.28447415e+01,  6.38857755e-03],
       ...,
       [ 3.95200344e-01,  2.07451536e+01,  7.36102701e+00, ...,
         3.35271042e+00,  3.92354873e-01,  2.36591661e+01],
       [-1.18201214e+00,  2.14270904e+01,  1.77436388e+01, ...,
         4.62177325e+00,  3.25702774e+01,  1.26910342e+01],
       [ 1.11094196e+01,  5.42387659e+01,  1.08160977e+01, ...,
         7.57029544e+00,  4.21289296e+00,  5.06826082e+00]])

In [64]:
scores_df = pd.DataFrame(reshaped_scores)
scores_df

,0,1,2,3,4,5,6,7,8,9,...,24497,24498,24499,24500,24501,24502,24503,24504,24505,24506
0,-2.833138,119.741751,37.343598,15.999384,15.547484,153.076439,44.964740,121.415133,6.113613,35.390153,...,56.137051,26.824418,2.785862,-2.768281,0.907981,10.367156,3.463433,13.496891,11.970038,25.751689
1,4.084168,-8.388913,9.718441,-1.381765,10.831382,35.616694,9.555389,23.712212,3.174904,-5.830132,...,12.741942,5.669044,-1.736879,9.855549,5.110872,3.412277,5.264712,-3.741695,-9.217974,1.842108
2,20.153912,100.362434,58.582506,57.461102,63.166478,342.710520,-inf,298.208294,17.533353,77.126699,...,-3.036495,-2.447193,16.751493,7.283449,-2.281001,7.196423,18.379143,-2.527445,12.844741,0.006389
3,51.843245,84.830725,60.147830,64.978518,44.775208,138.492274,74.593404,62.848948,18.993267,58.369592,...,18.864624,3.625798,38.103462,-10.272760,33.987429,12.868322,9.977123,29.818957,12.838145,-0.787574
4,13.286860,-5.556706,-7.605580,60.519830,-5.166604,37.646688,4.113645,86.213810,8.785900,8.630568,...,0.915110,1.015762,6.168036,4.259709,-2.272227,7.736611,11.336756,0.764597,-0.055255,-0.725600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41111,7.492072,24.180997,0.191121,-7.880123,21.160290,-3.162892,274.622391,28.856592,10.791275,1.215094,...,5.070986,31.809445,11.836491,3.353453,10.356299,2.276500,6.257723,6.368906,8.279688,8.616399
41112,1.760276,10.378586,12.814549,19.175799,2.597639,45.705647,-5.524238,16.114387,5.975273,13.585979,...,13.036463,-2.082533,-7.499100,8.588424,10.757933,10.868703,13.894020,-2.303291,8.856444,7.361128
41113,0.395200,20.745154,7.361027,2.682771,7.934245,-14.578211,-10.216811,7.472204,-4.778830,3.665278,...,5.743954,3.537149,8.981452,7.822079,2.572545,15.985165,0.372905,3.352710,0.392355,23.659166
41114,-1.182012,21.427090,17.743639,24.381523,2.891449,-25.659934,31.604494,22.166643,14.941262,10.067733,...,15.611279,1.209955,4.688934,0.675675,2.388437,-1.686501,-0.705218,4.621773,32.570277,12.691034


In [ ]:
#rec_df = pd.DataFrame(np_recs)
#rec_df.to_csv('besthybrid_recs.csv')

In [87]:
#scores_df.loc[0].argsort() #scores of items for user 0 (best is 575)
#max(scores_df.loc[0]) #max score is 731
#scores_df.loc[0]

0          575
1        16868
2         9960
3         2730
4         5068
         ...  
24502    18484
24503      391
24504       20
24505      752
24506      617
Name: 0, Length: 24507, dtype: int64

In [203]:
import time
import math

test_users = pd.read_csv('../RecSys2022/data/data_target_users_test.csv')
user_id = test_users['user_id']
recommendations = []
start_time = time.perf_counter()
for user in user_id:
    recommendations.append(recommender_h11.recommend(user,cutoff = 10))
    if math.trunc(time.perf_counter() - start_time) % 10 == 0:
        print(user)
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
191
192
194
195
196
197
198
199
201
202
203
204
205
206
207
208
209
210
211
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1341
1342
1343
1344
1345
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
22

In [ ]:
import winsound as ws
ws.Beep(300, 1000)

In [204]:
test_users.to_csv('BiasedSubmissions.csv', index=False)